In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
import re
import selenium
import pandas as pd
import html5lib
import math


In [194]:
"""
start_time = time.time()
url = 'https://www.ratemyprofessors.com/search/teachers?query=*&sid=1222'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)


driver.find_element_by_class_name('Buttons__Button-sc-19xdot-1 PaginationButton__StyledPaginationButton-txi1dr-1 eaZArN').click()

#hej=driver.find_element_by_class_name('SearchResultsPage__AddPromptWrapper-sc-1srop1v-2 gUWHGC').click()
#driver.find_element_by_class_name('CardName__StyledCardName-sc-1gyrgim-0 cJdVEK')
driver.find_element(By.linkText("Show More")).click()




#soup = BeautifulSoup(driver.page_source, 'lxml')
#company=soup.find('td',{'id':'domain_registrant_name'}).text.strip()
#print(company)
"""

'\nstart_time = time.time()\nurl = \'https://www.ratemyprofessors.com/search/teachers?query=*&sid=1222\'\ndriver = webdriver.Chrome(ChromeDriverManager().install())\ndriver.get(url)\n\n\ndriver.find_element_by_class_name(\'Buttons__Button-sc-19xdot-1 PaginationButton__StyledPaginationButton-txi1dr-1 eaZArN\').click()\n\n#hej=driver.find_element_by_class_name(\'SearchResultsPage__AddPromptWrapper-sc-1srop1v-2 gUWHGC\').click()\n#driver.find_element_by_class_name(\'CardName__StyledCardName-sc-1gyrgim-0 cJdVEK\')\ndriver.find_element(By.linkText("Show More")).click()\n\n\n\n\n#soup = BeautifulSoup(driver.page_source, \'lxml\')\n#company=soup.find(\'td\',{\'id\':\'domain_registrant_name\'}).text.strip()\n#print(company)\n'

In [195]:
"""
school_id=1222
number_of_professors=464
number_of_pages=math.ceil(number_of_professors/20)
school_url="https://www.ratemyprofessors.com/filter/professor/?&page=1&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid="+str(school_id)



#VIGTIG_URL="https://www.ratemyprofessors.com/filter/professor/?&page=1&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid=1222"
#20 PROFESSORER PR SIDE
#464 professors at Yale University

response = requests.get(school_url)
list_of_dicts=response.json() #keys are: ['professors', 'searchResultsTotal', 'remaining', 'type']
list_of_professor_dicts=list_of_dicts['professors'] 
number_of_professors=list_of_dicts['searchResultsTotal']

real_df=pd.concat([pd.DataFrame(list_of_dicts['professors'] [i], index=[i]) for i in range(20)],
          ignore_index=True)

"""

'\nschool_id=1222\nnumber_of_professors=464\nnumber_of_pages=math.ceil(number_of_professors/20)\nschool_url="https://www.ratemyprofessors.com/filter/professor/?&page=1&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid="+str(school_id)\n\n\n\n#VIGTIG_URL="https://www.ratemyprofessors.com/filter/professor/?&page=1&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid=1222"\n#20 PROFESSORER PR SIDE\n#464 professors at Yale University\n\nresponse = requests.get(school_url)\nlist_of_dicts=response.json() #keys are: [\'professors\', \'searchResultsTotal\', \'remaining\', \'type\']\nlist_of_professor_dicts=list_of_dicts[\'professors\'] \nnumber_of_professors=list_of_dicts[\'searchResultsTotal\']\n\nreal_df=pd.concat([pd.DataFrame(list_of_dicts[\'professors\'] [i], index=[i]) for i in range(20)],\n          ignore_index=True)\n\n'

The keys for each element in list_of_professor_dicts are:

'tDept' = Department (eg. science, spanish, law ect)

'tSid' = The school id (eg. 1222 for all Yale professors)

'institution_name' = Name of university

'tFname' = First name

'tMiddlename' = Middle name

'tLname' = Last name

'tid' = Teacher id

'tNumRatings' = Number of ratings

'rating_class' = How is their score (eg. good, average, poor, zero)

'contentType' = Have only seen this to return 'TEACHER'

'categoryType'= Have only seen this to return 'PROFESSOR'

'overall_rating' = Average rating (float)

In [2]:
def get_dataframe_of_school(school_id, number_of_professors):
    """This function returns a dataframe with al teachers for a given school as well as a bunch of attributes"""
    number_of_pages=math.ceil(number_of_professors/20)
    list_of_dfs=[]
    for j in range(1,number_of_pages+1):
        url=f"https://www.ratemyprofessors.com/filter/professor/?&page={j}&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid="+str(school_id)
        response = requests.get(url)
        list_of_dicts=response.json() #keys are: ['professors', 'searchResultsTotal', 'remaining', 'type']
        list_of_professor_dicts=list_of_dicts['professors'] 
        list_of_dfs.append(pd.concat([pd.DataFrame(list_of_dicts['professors'] [i], index=[i]) for i in range(len(list_of_professor_dicts))],
                  ignore_index=True))
    return pd.concat([list_of_dfs[i] for i in range(len(list_of_dfs))], axis= 0).reset_index(drop=True)

        
hej=get_dataframe_of_school(1222, 464)

In [3]:
def add_comments_to_df(df):
    list_of_teacher_ids=list(df.tid)
    list_of_comments_for_each_teacher=[]
    for i in range(len(list_of_teacher_ids)):
        print(i)
        my_url = f'https://www.ratemyprofessors.com/ShowRatings.jsp?tid={list_of_teacher_ids[i]}'
        my_response = requests.get(my_url)
        html = my_response.text
        soup = BeautifulSoup(html,'lxml')
        my_text = soup.findAll('div', {'class':'Comments__StyledComments-dzzyvm-0 gRjWel'})#list of comments unparsed
        finished_comments=[i.text for i in my_text]
        list_of_comments_for_each_teacher.append(finished_comments)
    df['Comments']=list_of_comments_for_each_teacher
        
    return df

ok=add_comments_to_df(hej)
ok

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,tDept,tSid,institution_name,tFname,tMiddlename,tLname,tid,tNumRatings,rating_class,contentType,categoryType,overall_rating,Comments
0,Law,1222,Yale University,Bruce,,Ackerman,839444,6,good,TEACHER,PROFESSOR,3.8,"[Like an earlier reviewer said, you will enjoy..."
1,Spanish,1222,Yale University,Rolena,,Adorno,1348108,3,good,TEACHER,PROFESSOR,4.0,"[Worth taking his class. A great teacher, Grea..."
2,Science,1222,Yale University,Charles,,Ahn,597242,5,average,TEACHER,PROFESSOR,2.6,[Fantastic teacher! Works hard and willing to ...
3,Mechanical Engineering,1222,Yale University,Charles,,Ahn,2099651,1,good,TEACHER,PROFESSOR,4.0,"[(to Moderator, class taken spring 2013) Prof...."
4,Psychology,1222,Yale University,Woo,,Ahn,2462350,0,zero,TEACHER,PROFESSOR,N/A,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,Theater,1222,Yale University,Grace,,Zandarski,2690272,0,zero,TEACHER,PROFESSOR,N/A,[]
460,Biology,1222,Yale University,Weiming,,Zhong,421797,2,average,TEACHER,PROFESSOR,2.8,"[Taught a difficult class, and I give him cred..."
461,Chemistry,1222,Yale University,Frederick,,Ziegler,582897,5,poor,TEACHER,PROFESSOR,1.3,[Worst professor I have ever had. I would leav...
462,Environmental Studies,1222,Yale University,Carl,,Zimmer,1375751,0,zero,TEACHER,PROFESSOR,N/A,[]


In [23]:
for i in range(50): print(ok['Comments'][i])

['Like an earlier reviewer said, you will enjoy this course and get a great deal out of it if you do the reading and come prepared to have a conversation. Professor Ackerman values different opinions and you will gain his respect, not lose it, if you disagree with his arguments but can raise even a reasonable defence of your position.', "He's fantastic if you do the reading. Like all good things in life, you only get as much out of his course as you're willing to put in.", 'he knows his material!', 'Arrogant, rambling, class adds nothing to just reading his book.', 'No Comments', 'No Comments']
['Worth taking his class. A great teacher', 'Great!', 'This was a very useful class, I am very interested in learn Spanish, because a lot of people in south america speak Spanish. In my home, we will start to practice spanglish.']
['Fantastic teacher! Works hard and willing to help if you initiate. Not the most open guy, but if you get to know him and spend time picking his brain about problems,

In [59]:
url_usl=f"https://www.usnews.com/best-colleges/rankings/national-universities?_mode=table"
#response = requests.get(url_usl)
#Button-suns4m-1 LoadMoreWrapper__LoadMore-zwyk5c-1 ipUrOX kmWga
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome(ChromeDriverManager().install())

#Go to all pages - 39 i alt
#for i in range(2):
 #   driver.get('https://www.usnews.com/best-colleges/rankings/national-universities?_page=' + str(i+1))
driver.get('https://www.usnews.com/best-colleges/rankings/national-universities?_page=1')
    
soup = BeautifulSoup(driver.page_source, 'lxml')

# counting results
clasees = soup.findAll('div', {'class':'Box-w0dun1-0 DetailCardColleges__TextContainer-cecerc-4 CYZBT eoCwZT'})
#print(len(teachers))

#print(clasees)


    
# counting results
#time.sleep(2)
#soup = BeautifulSoup(driver.page_source, 'lxml') teachers = soup.findAll('a', {'class':'TeacherCard__StyledTeacherCard-syjs0d-0 dLJIlx'})
#print(len(teachers))




====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\Liv Nøhr\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [82]:
#print(clasees[1])
#type(clasees[1])
#print(soup.split('="\00ab\00a0">')[1:])


#Få alle numre
numbers=[]
name=[]

driver = webdriver.Chrome(ChromeDriverManager().install())

#Få den første
for j in range(1):
    url = ('https://www.usnews.com/best-colleges/rankings/national-universities?_page=' + str(j+1))
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    clasees = soup.findAll('div', {'class':'Box-w0dun1-0 DetailCardColleges__TextContainer-cecerc-4 CYZBT eoCwZT'})
    class_name = soup.findAll('a', {'class':'Anchor-byh49a-0 DetailCardColleges__StyledAnchor-cecerc-7 fidpEI eKrerU card-name'})
    time.sleep(5)
    for i in range(len(clasees)):
        temp = clasees[i].text
        numbers.append(re.findall('\d+', temp)[0])
        name.append(class_name[i].text.strip())
    

print(numbers)
print(name)




====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\Liv Nøhr\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


['1', '2', '3', '4', '4', '6', '6', '8', '9', '9']
['Princeton University', 'Harvard University', 'Columbia University', 'Massachusetts Institute of Technology', 'Yale University', 'Stanford University', 'University of Chicago', 'University of Pennsylvania', 'California Institute of Technology', 'Johns Hopkins University']


In [83]:
#Få de sidste
driver = webdriver.Chrome(ChromeDriverManager().install())

url = ('https://www.usnews.com/best-colleges/rankings/national-universities?_page=28' )
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'lxml')
clasees = soup.findAll('div', {'class':'Box-w0dun1-0 DetailCardColleges__TextContainer-cecerc-4 CYZBT eoCwZT'})
class_name = soup.findAll('a', {'class':'Anchor-byh49a-0 DetailCardColleges__StyledAnchor-cecerc-7 fidpEI eKrerU card-name'})
for i in range(len(clasees)):
    temp = clasees[i].text
    numbers.append(re.findall('\d+', temp)[0])
    name.append(class_name[i].text.strip())
    

print(numbers)
print(name)
driver = webdriver.Chrome(ChromeDriverManager().install())


url = ('https://www.usnews.com/best-colleges/rankings/national-universities?_page=29')
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'lxml')
clasees = soup.findAll('div', {'class':'Box-w0dun1-0 DetailCardColleges__TextContainer-cecerc-4 CYZBT eoCwZT'})
class_name = soup.findAll('a', {'class':'Anchor-byh49a-0 DetailCardColleges__StyledAnchor-cecerc-7 fidpEI eKrerU card-name'})
for i in range(len(clasees)):
    temp = clasees[i].text
    numbers.append(re.findall('\d+', temp)[0])
    name.append(class_name[i].text.strip())
    
print(numbers)
print(name)
driver = webdriver.Chrome(ChromeDriverManager().install())

url = ('https://www.usnews.com/best-colleges/rankings/national-universities?_page=30')
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'lxml')
clasees = soup.findAll('div', {'class':'Box-w0dun1-0 DetailCardColleges__TextContainer-cecerc-4 CYZBT eoCwZT'})
class_name = soup.findAll('a', {'class':'Anchor-byh49a-0 DetailCardColleges__StyledAnchor-cecerc-7 fidpEI eKrerU card-name'})
for i in range(len(clasees)):
    temp = clasees[i].text
    numbers.append(re.findall('\d+', temp)[0])
    name.append(class_name[i].text.strip())
    
print(numbers)
print(name)
driver = webdriver.Chrome(ChromeDriverManager().install())

url = ('https://www.usnews.com/best-colleges/rankings/national-universities?_page=31')
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'lxml')
clasees = soup.findAll('div', {'class':'Box-w0dun1-0 DetailCardColleges__TextContainer-cecerc-4 CYZBT eoCwZT'})
class_name = soup.findAll('a', {'class':'Anchor-byh49a-0 DetailCardColleges__StyledAnchor-cecerc-7 fidpEI eKrerU card-name'})
for i in range(len(clasees)):
    temp = clasees[i].text
    numbers.append(re.findall('\d+', temp)[0])
    name.append(class_name[i].text.strip())
    

print(numbers)
print(name)
driver = webdriver.Chrome(ChromeDriverManager().install())

url = ('https://www.usnews.com/best-colleges/rankings/national-universities?_page=32')
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'lxml')
clasees = soup.findAll('div', {'class':'Box-w0dun1-0 DetailCardColleges__TextContainer-cecerc-4 CYZBT eoCwZT'})
class_name = soup.findAll('a', {'class':'Anchor-byh49a-0 DetailCardColleges__StyledAnchor-cecerc-7 fidpEI eKrerU card-name'})
for i in range(len(clasees)):
    temp = clasees[i].text
    numbers.append(re.findall('\d+', temp)[0])
    name.append(class_name[i].text.strip())
    

print(numbers)
print(name)
driver = webdriver.Chrome(ChromeDriverManager().install())

url = ('https://www.usnews.com/best-colleges/rankings/national-universities?_page=33')
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'lxml')
clasees = soup.findAll('div', {'class':'Box-w0dun1-0 DetailCardColleges__TextContainer-cecerc-4 CYZBT eoCwZT'})
class_name = soup.findAll('a', {'class':'Anchor-byh49a-0 DetailCardColleges__StyledAnchor-cecerc-7 fidpEI eKrerU card-name'})
for i in range(len(clasees)):
    temp = clasees[i].text
    numbers.append(re.findall('\d+', temp)[0])
    name.append(class_name[i].text.strip())
    

print(numbers)
print(name)
driver = webdriver.Chrome(ChromeDriverManager().install())

url = ('https://www.usnews.com/best-colleges/rankings/national-universities?_page=34')
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'lxml')
clasees = soup.findAll('div', {'class':'Box-w0dun1-0 DetailCardColleges__TextContainer-cecerc-4 CYZBT eoCwZT'})
class_name = soup.findAll('a', {'class':'Anchor-byh49a-0 DetailCardColleges__StyledAnchor-cecerc-7 fidpEI eKrerU card-name'})
for i in range(len(clasees)):
    temp = clasees[i].text
    numbers.append(re.findall('\d+', temp)[0])
    name.append(class_name[i].text.strip())
    
    

print(numbers)
print(name)
driver = webdriver.Chrome(ChromeDriverManager().install())

url = ('https://www.usnews.com/best-colleges/rankings/national-universities?_page=35')
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'lxml')
clasees = soup.findAll('div', {'class':'Box-w0dun1-0 DetailCardColleges__TextContainer-cecerc-4 CYZBT eoCwZT'})
class_name = soup.findAll('a', {'class':'Anchor-byh49a-0 DetailCardColleges__StyledAnchor-cecerc-7 fidpEI eKrerU card-name'})
for i in range(len(clasees)):
    temp = clasees[i].text
    numbers.append(re.findall('\d+', temp)[0])
    name.append(class_name[i].text.strip())
    
    

print(numbers)
print(name)
driver = webdriver.Chrome(ChromeDriverManager().install())

url = ('https://www.usnews.com/best-colleges/rankings/national-universities?_page=36')
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'lxml')
clasees = soup.findAll('div', {'class':'Box-w0dun1-0 DetailCardColleges__TextContainer-cecerc-4 CYZBT eoCwZT'})
class_name = soup.findAll('a', {'class':'Anchor-byh49a-0 DetailCardColleges__StyledAnchor-cecerc-7 fidpEI eKrerU card-name'})
for i in range(len(clasees)):
    temp = clasees[i].text
    numbers.append(re.findall('\d+', temp)[0])
    name.append(class_name[i].text.strip())
    
    

print(numbers)
print(name)
driver = webdriver.Chrome(ChromeDriverManager().install())

url = ('https://www.usnews.com/best-colleges/rankings/national-universities?_page=37')
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'lxml')
clasees = soup.findAll('div', {'class':'Box-w0dun1-0 DetailCardColleges__TextContainer-cecerc-4 CYZBT eoCwZT'})
class_name = soup.findAll('a', {'class':'Anchor-byh49a-0 DetailCardColleges__StyledAnchor-cecerc-7 fidpEI eKrerU card-name'})
for i in range(len(clasees)):
    temp = clasees[i].text
    numbers.append(re.findall('\d+', temp)[0])
    name.append(class_name[i].text.strip())
    

print(numbers)
print(name)
driver = webdriver.Chrome(ChromeDriverManager().install())

url = ('https://www.usnews.com/best-colleges/rankings/national-universities?_page=38')
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'lxml')
clasees = soup.findAll('div', {'class':'Box-w0dun1-0 DetailCardColleges__TextContainer-cecerc-4 CYZBT eoCwZT'})
class_name = soup.findAll('a', {'class':'Anchor-byh49a-0 DetailCardColleges__StyledAnchor-cecerc-7 fidpEI eKrerU card-name'})
for i in range(len(clasees)):
    temp = clasees[i].text
    numbers.append(re.findall('\d+', temp)[0])
    name.append(class_name[i].text.strip())
    
    

print(numbers)
print(name)
driver = webdriver.Chrome(ChromeDriverManager().install())

url = ('https://www.usnews.com/best-colleges/rankings/national-universities?_page=39')
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'lxml')
clasees = soup.findAll('div', {'class':'Box-w0dun1-0 DetailCardColleges__TextContainer-cecerc-4 CYZBT eoCwZT'})
class_name = soup.findAll('a', {'class':'Anchor-byh49a-0 DetailCardColleges__StyledAnchor-cecerc-7 fidpEI eKrerU card-name'})
for i in range(len(clasees)):
    temp = clasees[i].text
    numbers.append(re.findall('\d+', temp)[0])
    name.append(class_name[i].text.strip())
    

print(numbers)
print(name)






====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\Liv Nøhr\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\Liv Nøhr\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


['1', '2', '3', '4', '4', '6', '6', '8', '9', '9', '272', '272', '272', '272', '272', '272', '272', '272', '272', '272']
['Princeton University', 'Harvard University', 'Columbia University', 'Massachusetts Institute of Technology', 'Yale University', 'Stanford University', 'University of Chicago', 'University of Pennsylvania', 'California Institute of Technology', 'Johns Hopkins University', 'Dallas Baptist University', 'Florida Atlantic University', 'Long Island University', 'North Carolina Agricultural and Technical State University', 'Sam Houston State University', 'Tennessee Technological University', 'University of Akron', 'University of Alabama--Huntsville', 'University of Missouri--Kansas City', 'University of North Florida']




====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\Liv Nøhr\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


['1', '2', '3', '4', '4', '6', '6', '8', '9', '9', '272', '272', '272', '272', '272', '272', '272', '272', '272', '272', '272', '272', '284', '284', '284', '284', '284', '284', '284', '284']
['Princeton University', 'Harvard University', 'Columbia University', 'Massachusetts Institute of Technology', 'Yale University', 'Stanford University', 'University of Chicago', 'University of Pennsylvania', 'California Institute of Technology', 'Johns Hopkins University', 'Dallas Baptist University', 'Florida Atlantic University', 'Long Island University', 'North Carolina Agricultural and Technical State University', 'Sam Houston State University', 'Tennessee Technological University', 'University of Akron', 'University of Alabama--Huntsville', 'University of Missouri--Kansas City', 'University of North Florida', 'University of the Incarnate Word', 'William Carey University', 'Ball State University', 'Delaware State University', "D'Youville College", 'Gardner-Webb University', 'Indiana University 



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515


...

['1', '2', '3', '4', '4', '6', '6', '8', '9', '9', '272', '272', '272', '272', '272', '272', '272', '272', '272', '272', '272', '272', '284', '284', '284', '284', '284', '284', '284', '284', '284', '284', '284', '284', '284', '284', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298']
['Princeton University', 'Harvard University', 'Columbia University', 'Massachusetts Institute of Technology', 'Yale University', 'Stanford University', 'University of Chicago', 'University of Pennsylvania', 'California Institute of Technology', 'Johns Hopkins University', 'Dallas Baptist University', 'Florida Atlanti



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\Liv Nøhr\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


['1', '2', '3', '4', '4', '6', '6', '8', '9', '9', '272', '272', '272', '272', '272', '272', '272', '272', '272', '272', '272', '272', '284', '284', '284', '284', '284', '284', '284', '284', '284', '284', '284', '284', '284', '284', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298']
['Princeton University', 'Harvard University', 'Columbia University', 'Massachusetts Institute of Technology', 'Yale University', 'Stanford University', 'University of Chicago', 'University of Pennsylvania', 'California Institute of Technology', 'Jo



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\Liv Nøhr\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


['1', '2', '3', '4', '4', '6', '6', '8', '9', '9', '272', '272', '272', '272', '272', '272', '272', '272', '272', '272', '272', '272', '284', '284', '284', '284', '284', '284', '284', '284', '284', '284', '284', '284', '284', '284', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298', '298']
['Princeton University', 'Harvard University', 'Columbia University', 'Massachusetts Institute of Technology', 'Yale University', 'Stanford University', 'University of Chicago', '

In [94]:
df = pd.DataFrame(list(zip(name, numbers)),columns =['Name', 'Number'])

df_last = df[df['Number']=='298']
df_last.sample(n = 10)

,Name,Number
123,Wichita State University,298
122,Western Kentucky University,298
98,University of Charleston,298
77,Regent University,298
100,University of Hawaii--Hilo,298
47,Cleveland State University,298
55,Husson University,298
75,Palm Beach Atlantic University,298
89,Texas Woman's University,298
114,University of Texas at Tyler,298
